In [ ]:

# Import python packages
import streamlit as st

from snowflake.core import Root

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session

session = get_active_session()
root = Root(session)
session.query_tag = {"origin":"sf_sit-is",
                     "name":"entity_resolution",
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}

db = session.get_current_database().strip('"')
wh = session.get_current_warehouse().strip('"')
prefix = session.get_current_warehouse().strip('"').split("_DS_WH")[0]
     

# sql_command = f"ALTER WAREHOUSE {wh} SET WAREHOUSE_SIZE = 'rge'"
# session.sql(sql_command).collect()
     

session.sql(f"USE ROLE ACCOUNTADMIN").collect()
session.sql(f"CREATE SCHEMA IF NOT EXISTS {db}.MATCH").collect()
session.sql(f"USE SCHEMA {db}.MATCH").collect()


In [ ]:
CREATE OR REPLACE TABLE MDM_TRAINING_SET_VALID AS

select
    UUID_STRING() id,
    1::boolean IS_VALID,
    c.SOURCE_PKEY,
    c.NAME,
    c.SOURCE_SYSTEM,
    ca.ADDRESS_LINE_1,
    ca.ADDRESS_LINE_2,
    ca.CITY,
    ca.STATE,
    ca.POSTAL_CODE,
    ca.COUNTRY,
    CONCAT(NAME, ' ',ADDRESS_LINE_1, ' ',ADDRESS_LINE_2, ' ',CITY, ' ',ca.STATE, ' ',POSTAL_CODE) CUSTOMER_FULL_DETAIL,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', CUSTOMER_FULL_DETAIL) AS CUSTOMER_FULL_DETAIL_EMBEDDING
from
    hmh_prd_master_data.raw.customer c
    inner join HMH_PRD_MASTER_DATA.CUSTOMER.CUSTOMER_ADDRESS ca on
        c.BUSINESS_ID = ca.CUSTOMER_BUSINESS_ID
        and c.source_system = ca.SOURCE_SYSTEM

;

In [ ]:
CREATE OR REPLACE TABLE MDM_TRAINING_SET_INVALID AS
select 
    UUID_STRING() id,
    0::boolean IS_VALID,
    cx.SOURCE_PKEY,
    cx.NAME,
    cx.SOURCE_SYSTEM,
    ca.ADDRESS_LINE_1,
    ca.ADDRESS_LINE_2,
    ca.CITY,
    ca.STATE,
    ca.POSTAL_CODE,
    ca.COUNTRY,
    CONCAT(NAME, ' ',ADDRESS_LINE_1, ' ',ADDRESS_LINE_2, ' ',CITY, ' ',ca.STATE, ' ',POSTAL_CODE) CUSTOMER_FULL_DETAIL,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', CUSTOMER_FULL_DETAIL) AS CUSTOMER_FULL_DETAIL_EMBEDDING

from
    hmh_prd_master_data.raw.customer_xref cx
    inner join HMH_PRD_MASTER_DATA.CUSTOMER.CUSTOMER_ADDRESS_XREF ca on
        cx.SOURCE_PKEY = ca.CUSTOMER_BUSINESS_ID
        and cx.source_system = ca.SOURCE_SYSTEM
where cx.source_system != 'b360.default.system'
  and cx.state = 'ACTIVE'
  and business_id in (
    select business_id from hmh_prd_master_data.raw.customer_xref where source_system != 'b360.default.system' and state = 'ACTIVE' group by business_id having count(*) = 1)

In [ ]:
--Select an address from the invalid set and see if it matches one in the valid.
select 
VECTOR_COSINE_SIMILARITY(i.CUSTOMER_FULL_DETAIL_EMBEDDING, v.CUSTOMER_FULL_DETAIL_EMBEDDING),
i.id,
v.customer_full_Detail VALID,
i.customer_full_detail INVALID
from MDM_TRAINING_SET_INVALID i,
MDM_TRAINING_SET_VALID v
where 1=1
and i.id = 'ba1d6755-02ad-47f6-86a1-570175e8240f' 
-- and i.ID in (
-- 'ffde6696-6ee4-4150-8a7e-6c938de88598',
-- '10b0bbe8-b477-44fb-ba36-0714d38400b7',
-- '65617048-8391-4961-b758-1896da73ddfa',
-- '1f3d5f35-3786-4fe6-a170-0e05bbb6caae',
-- '7ba569e8-8469-4d67-8db4-f158b78bb595',
-- '13e179ed-8e7a-46fe-9318-5bfbc199f5cd'
-- )
order by 1 desc
;